In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.linalg as la
import json
from scipy.integrate import solve_ivp

np.set_printoptions(precision=5, suppress=True)
pd.set_option("display.precision", 15)
np.set_printoptions(formatter={'float': lambda x: "{0:0.7f}".format(x)})
pd.set_option('display.float_format', lambda x: "{0:0.7f}".format(x))
pd.set_option('display.max_columns', None)

\begin{align}
\dot C(t) &= -K_{ch} C(t)H(t) - K_{ce} C(t)E(t) \\
\dot H(t) &= K_{ch} C(t)H(t) - K_{he} H(t)E(t) \\
\dot E(t) &= (K_{ce} C(t)E(t) + K_{he} H(t)E(t) + \alpha S(t)) (1 - E(t)) \\
\dot S(t) &= \beta S(t)(1 - \frac{S(t)}{C_s})
\end{align}

In [2]:
# load the charging data
charging_df = pd.read_csv("data/charging.csv")
display(charging_df)

# load the sales data
sales_df = pd.read_csv("data/vehicle_sales_2000_2023.csv")
sales_df = sales_df.merge(charging_df, on='Year', how='left')
sales_df['Charging Ports'] = sales_df['Charging Ports'].fillna(0).astype(float) / 1000
sales_df['Station Locations'] = sales_df['Station Locations'].fillna(0).astype(float) / 1000
sales_df['Station Locations Ratio'] = sales_df['Station Locations'] / sales_df['Total']
display(sales_df)

rel_sales_df = sales_df.loc[sales_df['Year'] >= 2011, :].copy()
display(rel_sales_df)

,Year,Charging Ports,Station Locations
0,2007,417,139
1,2008,564,196
2,2009,771,259
3,2010,1256,407
4,2011,5248,2109
5,2012,10726,5444
6,2013,16619,6938
7,2014,22470,9207
8,2015,26532,10710
9,2016,33165,13150


,Year,Combustion,Hybrid,Electric,Total,Combustion_ratio,Hybrid_ratio,Electric_ratio,Charging Ports,Station Locations,Station Locations Ratio
0,2000,17349.7000000,10,0,17359.7000000,0.9994240,0.0005760,0.0000000,0.0000000,0.0000000,0.0000000
1,2001,17122.4000000,20,0,17142.4000000,0.9988333,0.0011667,0.0000000,0.0000000,0.0000000,0.0000000
2,2002,16816.2000000,40,0,16856.2000000,0.9976270,0.0023730,0.0000000,0.0000000,0.0000000,0.0000000
3,2003,16639.1000000,50,0,16689.1000000,0.9970040,0.0029960,0.0000000,0.0000000,0.0000000,0.0000000
4,2004,16866.9000000,90,0,16956.9000000,0.9946924,0.0053076,0.0000000,0.0000000,0.0000000,0.0000000
5,2005,16948.2000000,210,0,17158.2000000,0.9877610,0.0122390,0.0000000,0.0000000,0.0000000,0.0000000
6,2006,16504.1000000,250,0,16754.1000000,0.9850783,0.0149217,0.0000000,0.0000000,0.0000000,0.0000000
7,2007,16089.0000000,350,0,16439.0000000,0.9787092,0.0212908,0.0000000,0.4170000,0.1390000,0.0000085
8,2008,13194.8000000,310,0,13504.8000000,0.9770452,0.0229548,0.0000000,0.5640000,0.1960000,0.0000145
9,2009,10402.3000000,290,0,10692.3000000,0.9728777,0.0271223,0.0000000,0.7710000,0.2590000,0.0000242


,Year,Combustion,Hybrid,Electric,Total,Combustion_ratio,Hybrid_ratio,Electric_ratio,Charging Ports,Station Locations,Station Locations Ratio
11,2011,12741.8000000,260,10,13011.8000000,0.9792496,0.0199819,0.0007685,5.2480000,2.1090000,0.0001621
12,2012,14433.2000000,460,15,14908.2000000,0.9681383,0.0308555,0.0010062,10.7260000,5.4440000,0.0003652
13,2013,15530.1000000,540,40,16110.1000000,0.9639977,0.0335193,0.0024829,16.6190000,6.9380000,0.0004307
14,2014,16452.2000000,510,60,17022.2000000,0.9665143,0.0299609,0.0035248,22.4700000,9.2070000,0.0005409
15,2015,17408.0000000,430,70,17908.0000000,0.9720795,0.0240116,0.0039089,26.5320000,10.7100000,0.0005981
16,2016,17477.3000000,420,90,17987.3000000,0.9716467,0.0233498,0.0050035,33.1650000,13.1500000,0.0007311
17,2017,17150.1000000,450,105,17705.1000000,0.9686531,0.0254164,0.0059305,45.7890000,16.1700000,0.0009133
18,2018,17224.9000000,460,205,17889.9000000,0.9628282,0.0257128,0.0114590,56.8420000,19.8930000,0.0011120
19,2019,16961.1000000,490,230,17681.1000000,0.9592786,0.0277132,0.0130082,73.8380000,23.2820000,0.0013168
20,2020,14471.8000000,530,240,15241.8000000,0.9494810,0.0347728,0.0157462,96.1900000,28.6020000,0.0018766


In [5]:
C_s = 168 / 15194.3
print(C_s)

0.011056777870648862


In [6]:
from scipy.optimize import fsolve

In [11]:
def ces(t, y, K_ch, K_ce, K_he, alpha, beta, C_s):
    #print(y[0], y[1], y[2], y[3], C_s, y[3] / C_s)
    return (
        -K_ch * y[0] * y[1] - K_ce * y[0] * y[2],
        K_ch * y[0] * y[1] - K_he * y[1] * y[2],
        (K_ce * y[0] * y[2] + K_he * y[1] * y[2] + alpha * y[3]) * (1-y[2]),
        beta * y[3] * (1 - y[3] / C_s)
    )
    
t_span = (0, 50-11)
ts = np.linspace(*t_span, 500)

ts_int = ts.astype(int)
ts_yr_inds = np.where(ts_int[1:] != ts_int[:-1])[0] + 1
ts_yr_inds = np.concatenate([[0], ts_yr_inds])

mn_yr, mx_yr = rel_sales_df['Year'].min(), rel_sales_df['Year'].max()
ts_2011 = (ts[ts_yr_inds] + 2011).astype(int)
inner_years = (ts_2011 >= mn_yr) & (ts_2011 <= mx_yr)
inner_ts_yr_inds = ts_yr_inds[inner_years]

C = rel_sales_df['Combustion_ratio'].values
H = rel_sales_df['Hybrid_ratio'].values
E = rel_sales_df['Electric_ratio'].values
S = rel_sales_df['Station Locations Ratio'].values


y0 = np.array([
    rel_sales_df['Combustion_ratio'].values[0],
    rel_sales_df['Hybrid_ratio'].values[0],
    rel_sales_df['Electric_ratio'].values[0],
    rel_sales_df['Station Locations Ratio'].values[0]
])

C_s = 168 / rel_sales_df['Total'].values[0]
print(C_s)

x = ['K_ch', 'K_ce', 'K_he', 'alpha', 'beta']
x0 = np.array([0.1, 0.1, 0.01, 4.0, 0.3])

def h(x, C, H, E, S):
    K_ch, K_ce, K_he, alpha, beta = x
    global C_s
    solution = solve_ivp(ces, t_span, y0, t_eval=ts, args=(K_ch, K_ce, K_he, alpha, beta, C_s))
    
    model_C = solution.y[0]
    model_H = solution.y[1]
    model_E = solution.y[2]
    model_S = solution.y[3]
    
    C_ = model_C[inner_ts_yr_inds]
    H_ = model_H[inner_ts_yr_inds]
    E_ = model_E[inner_ts_yr_inds]
    S_ = model_S[inner_ts_yr_inds]
    
    diff = (
        (C - C_) + 
        (H - H_) + 
        (E - E_) +
        (S - S_))
    
    avg1 = np.mean(diff[:4])
    avg2 = np.mean(diff[4:6])
    avg3 = np.mean(diff[6:8])
    avg4 = np.mean(diff[8:10])
    avg5 = np.mean(diff[10:])
    res_diff = np.array([avg1, avg2, avg3, avg4, avg5])
    
    print(res_diff)
    
    # find how close the model is to the data
    return res_diff
    
params = fsolve(h, x0, args=(C, H, E, S))
print(params)

K_ch, K_ce, K_he, alpha, beta = params

    


    

0.012911357383298237
[-0.0013334 -0.0061353 -0.0127151 -0.0241213 -0.0409669]
[-0.0013334 -0.0061353 -0.0127151 -0.0241213 -0.0409669]
[-0.0013334 -0.0061353 -0.0127151 -0.0241213 -0.0409669]
[-0.0013334 -0.0061353 -0.0127151 -0.0241213 -0.0409669]
[-0.0013334 -0.0061353 -0.0127151 -0.0241213 -0.0409669]
[-0.0013334 -0.0061353 -0.0127151 -0.0241213 -0.0409669]
[-0.0013334 -0.0061353 -0.0127151 -0.0241213 -0.0409669]
[-0.0013334 -0.0061353 -0.0127151 -0.0241213 -0.0409669]


IndexError: index 39 is out of bounds for axis 0 with size 34